## Car Pooling Algorithm




In [26]:
from pandas import reset_option
import random
import requests
import copy

### Graph

In [4]:
class node:
    def __init__(self, id, type, flag, lat, long):
        self.id = id
        self.type = type # 0: car, 1: start, 2:end
        self.isAccessible = flag
        self.lat = lat
        self.long = long

In [5]:
class edge:
    def __init__(self, id, startNodeId, endNodeId, weight):
        self.id = id
        self.startNodeId = startNodeId
        self.endNodeId = endNodeId
        self.weight = weight

In [6]:
class graph:
    def __init__(self):
        self.adjList = [[]]
        self.nodes = []
        self.nodeCounter = 0
        self.edgeCounter = 0
    
    def getNodeFromId(self, nodeId):
        for n in self.nodes:
            if n.id == nodeId:
                return n
    
    def getEdgeFromEdgeNodes(self, node1, node2):
        for e in self.adjList[node1]:
            if e.endNodeId == node2:
                return e
        return -1
    
    def addNode(self, type, flag, lat, long):
        self.nodes.append(node(self.nodeCounter, type, flag, lat, long))
        self.nodeCounter += 1
        self.adjList.append([])
    
    def addEdge(self, startNode, endNode, weight):
        self.adjList[startNode].append(edge(self.edgeCounter, startNode, endNode, weight))
        self.edgeCounter += 1
    
    def calcEdgeWeight(self, startNode, endNode):
        s = self.getNodeFromId(startNode)
        e = self.getNodeFromId(endNode)

        r = requests.get('http://127.0.0.1:5000/route/v1/driving/'+ str(s.long) +','+ str(s.lat) +';'+ str(e.long) +','+ str(e.lat) +'?steps=false')
        data = r.json()

        return data['routes'][0]['distance']
    
    def addCar(self, loc):
        self.addNode(0, False, loc[1], loc[0])

        for node in self.nodes:
            if node.type == 1:
                self.addEdge(self.nodeCounter-1, node.id, self.calcEdgeWeight(self.nodeCounter-1, node.id))

    def addTrip(self, start, end):
        self.addNode(1, True, start[1], start[0]) # Check the correct order for lat and long with OSRM
        self.addNode(2, False, end[1], end[0])

        for node in self.nodes:
            if node.id != self.nodeCounter-1 and node.id != self.nodeCounter-2: # add an edge from every node to the new start and end nodes 
                self.addEdge(node.id, self.nodeCounter-2, self.calcEdgeWeight(node.id, self.nodeCounter-2))
                if node.type != 0:
                    self.addEdge(node.id, self.nodeCounter-1, self.calcEdgeWeight(node.id, self.nodeCounter-1))

            if node.type != 0 and node.id != self.nodeCounter-2: # add an edge from the new start node to all the other start and end nodes
                self.addEdge(self.nodeCounter-2, node.id, self.calcEdgeWeight(self.nodeCounter-2, node.id))

            if node.type != 0 and node.id != self.nodeCounter-1 and node.id != self.nodeCounter-2: # add an edge from the new end node to all the other end and start nodes
                self.addEdge(self.nodeCounter-1, node.id, self.calcEdgeWeight(self.nodeCounter-1, node.id))
    
    # Functions for testing 

    def printAdjList(self):
        for x in range(0, len(self.adjList)):
            print(x, ": ", end =" ")
            for y in self.adjList[x]:
                print(y.endNodeId, end =" ")
            print("")

### OSRM

In [23]:
pickupNetwork = graph()
# enter coords as long, lat
pickupNetwork.addCar([14.513809277460041, 35.89897453256716]) # valletta

pickupNetwork.addTrip([14.423235598020154, 35.91419450996914], [14.407218690503381, 35.888194056331706]) # Mosta to imdina
pickupNetwork.addTrip([14.49291350433241, 35.87369410066685], [14.513809277460041, 35.89897453256716]) # Marsa to Valletta
pickupNetwork.addTrip([14.349747452527506, 35.952589620545496], [14.488425821564382, 35.88613649037252])  # Mellieha to Hamrun


### Tabu Search

In [7]:
def fitnessFunc(solution, graph):
    value = 0
    for n in range(1, len(solution)):
        n1 = solution[n]
        n2 = solution[n-1]
        value += graph.getEdgeFromEdgeNodes(n2, n1).weight # adds up the weight of all the edges in the path
    
    return value

creating a random solution 

In [25]:
def initSolution(graph): # used to randomly generate an solution 
    solution = []
    visited = [False] * len(graph.nodes) # will indicate if a node has been visited yet

    for n in graph.nodes:
        if n.type == 0: # finds a car as the starting point
            solution.append(n.id)
            visited[n.id] = True
    
    while(visited.count(visited[0]) != len(visited)): # while the values in the visited list are not all equal
        avalabile = False 
        n = random.choice(graph.adjList[solution[-1]]).endNodeId # picks a random node that can be reached form the current last node in the solution
        if graph.getNodeFromId(n).type == 2: # if the node picked is a end point 
            start = n -1
            for x in solution: # check if its start point is already in the list 
                if x == start:
                    avalabile = True # if yes we can add the point to the solution 
                    break
        else:
            avalabile = True

        if visited[n] == False and avalabile: # add the point to the solution 
            visited[n] = True
            solution.append(n)

    return solution

neighborhood function

In [26]:
def neighborhood(solution, graph): # returns all the possible swaps for a given solution 
    neighborhood = []

    for x in range(1, len(solution)):
        for y in range(x + 1, len(solution)):
            newSolution = solution.copy()
            newSolution[x], newSolution[y] = newSolution[y], newSolution[x] # swap 2 values 
            if checkSolution(newSolution, graph) == True: # check if swap is valid
                neighborhood.append(newSolution) # if swap is good add it to the list 
    
    return neighborhood


In [19]:
def checkSolution(solution, graph):
    correct = True
    # loop over the whole solution and check if it is possible to get from one node to the next. If it is solution is valid
    for n in range(1, len(solution)): 
        n1 = solution[n]
        n2 = solution[n-1]
        edge = graph.getEdgeFromEdgeNodes(n2, n1) # returns -1 if there is no edge

        if edge == -1:
            return False # the solution is not valid 

        if graph.getNodeFromId(n1).type == 2: # if the point is an end point 
            start = n1 -1
            for x in solution[:n]: # check if its start point is in the list before it
                if x == start:
                    correct = True
                    break
                else:
                    correct = False

        if not correct:
            return False

    return True


### Tabu Search

In [28]:
def tabuSearch(graph, iterations, tabuSize, s):
    bestSolution = s
    solution = bestSolution
    tabuList = list()
    bestCost = fitnessFunc(bestSolution, graph)
    counter = 1

    while counter <= iterations:
        neighbours = neighborhood(solution, graph)
        found = False # indicates we found a set of different nodes that are not in the tabu list 
        currentBestSolutionIndex = 0
        currentBestSolution = neighbours[currentBestSolutionIndex]

        while not found and currentBestSolutionIndex < len(neighbours) - 1:
            i = 0
            while i < len(currentBestSolution):
                if currentBestSolution[i] != solution[i]: # if the same node position is not equal 
                    firstNode = currentBestSolution[i]
                    secondNode = solution[i]
                    break
                i = i + 1

            if [firstNode, secondNode] not in tabuList and [secondNode, firstNode] not in tabuList: # check if the swap is in the tabu list or not 
                tabuList.append([firstNode, secondNode]) # add the set to the tabuList
                found = True 

                solution = currentBestSolution
                cost = fitnessFunc(currentBestSolution, graph)

                if cost < bestCost: # checks if the new solution is better than the current best 
                    bestCost = cost
                    bestSolution = solution
            
            else:
                currentBestSolutionIndex += 1 # if nothing was found in with the current swap, go the the next one 
                currentBestSolution = neighbours[currentBestSolutionIndex]
        
        if len(tabuList) >= tabuSize:
            tabuList.pop(0) # removes the oldest element in the list

        counter = counter + 1
    
    return bestSolution, bestCost



In [29]:
s = initSolution(pickupNetwork)

In [30]:
print(s)

[0, 5, 1, 6, 3, 2, 4]


In [35]:
bestSolution, bestCost = tabuSearch(pickupNetwork, 500000, 7, s)

In [36]:
print("Start solution: ", s, " Cost: ", fitnessFunc(s, pickupNetwork))
print("End solution: ", bestSolution, " Cost: ", bestCost)

Start solution:  [0, 5, 1, 6, 3, 2, 4]  Cost:  65593.6
End solution:  [0, 5, 1, 2, 6, 3, 4]  Cost:  54725.9


### using multiple cars

In [53]:
pickupNetwork2 = graph()
# enter coords as long, lat
pickupNetwork2.addCar([14.513809277460041, 35.89897453256716]) # valletta
pickupNetwork2.addCar([14.513809277460041, 35.89897453256716]) # valletta
pickupNetwork2.addCar([14.513809277460041, 35.89897453256716]) # valletta

pickupNetwork2.addTrip([14.423235598020154, 35.91419450996914], [14.407218690503381, 35.888194056331706]) # Mosta to mdina
pickupNetwork2.addTrip([14.49291350433241, 35.87369410066685], [14.513809277460041, 35.89897453256716]) # Marsa to Valletta
pickupNetwork2.addTrip([14.349747452527506, 35.952589620545496], [14.488425821564382, 35.88613649037252])  # Mellieha to Hamrun



pickupNetwork2.addTrip([14.396021566820055, 35.93589926856759], [14.407218690503381, 35.888194056331706]) # st pauls bay to mdina
pickupNetwork2.addTrip([14.513809277460041, 35.89897453256716], [14.396021566820055, 35.93589926856759]) # Valletta to st pauls bay
pickupNetwork2.addTrip([14.469391291395608,35.87691014067125], [14.488425821564382, 35.88613649037252])  # qormi to Hamrun


pickupNetwork2.addTrip([14.432798599308622, 35.846482945229674], [14.480448112126183, 35.8215269171298]) # siggiewi to zurrieq
pickupNetwork2.addTrip([14.480448112126183, 35.8215269171298], [14.432798599308622, 35.846482945229674]) # zurrieq to siggiewi
pickupNetwork2.addTrip([14.349747452527506, 35.952589620545496], [14.510671636760655, 35.88200443585789])  # Mellieha to paola


In [54]:
carSize = 3
def initSolution2(graph): # used to randomly generate a solution 
    solutions = []
    visited = [False] * len(graph.nodes) # will indicate if a node has been visited yet
    
    nodePointer = 0


    for n in graph.nodes:
        solution = []
        if n.type == 0: # finds a car as the starting point
            solution.append(n.id)
            visited[n.id] = True
            s = 0
            while s < carSize:
                if graph.getNodeFromId(nodePointer).type != 0:
                    solution.append(nodePointer)
                    solution.append(nodePointer + 1)

                    nodePointer += 2
                    s += 1
                else:
                    nodePointer += 1

            solutions.append(solution)

    return solutions

In [87]:
solutions = initSolution2(pickupNetwork2)

print(solutions)


[[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]]


In [96]:
def neighborhood2(solutions, graph): # returns all the possible swaps for a given solution 
    neighborhood = []

    for s in range(len(solutions)):
        for x in range(1, len(solutions[s])):
            for y in range(x + 1, len(solutions[s])):
                newSolution = copy.deepcopy(solutions)
                newSolution[s][x], newSolution[s][y] = newSolution[s][y], newSolution[s][x] # swap 2 values 
                if checkSolution(newSolution[s], graph) == True: # check if swap is valid
                    neighborhood.append(newSolution) # if swap is good add it to the list 
        
    for s in range(len(solutions)):
        for pos in range(1, len(solutions[s])):
            if graph.getNodeFromId(solutions[s][pos]).type == 1:
                start = pos
                end = solutions[s].index(graph.getNodeFromId(solutions[s][pos]+1).id)

                for sol in range(s+1, len(solutions)):
                    for index in range(1, len(solutions[s])):
                        if graph.getNodeFromId(solutions[sol][index]).type == 1:
                            newSolution = copy.deepcopy(solutions)
                            newSolution[s][start], newSolution[s][end] = newSolution[sol][index], newSolution[sol][solutions[sol].index(graph.getNodeFromId(solutions[sol][index]+1).id)]
                            neighborhood.append(newSolution)


    
    return neighborhood

In [57]:
def fitnessFunc2(solutions, graph):
    value = 0
    for solution in solutions:
        for n in range(1, len(solution)):
            n1 = solution[n]
            n2 = solution[n-1]
            value += graph.getEdgeFromEdgeNodes(n2, n1).weight # adds up the weight of all the edges in the path
    
    return value

In [97]:
n = neighborhood2(solutions, pickupNetwork2)

print(n)

[[[0, 3, 5, 4, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 7, 5, 6, 4, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 4, 5, 7, 6, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 11, 10, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 13, 11, 12, 10, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 13, 12, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 17, 16, 18, 19, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 19, 17, 18, 16, 20]], [[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 19, 18, 20]], [[0, 9, 10, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 11, 12, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 13, 14, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]], [[0, 15, 1

In [98]:
def tabuSearch2(graph, iterations, tabuSize, s):
    bestSolution = s
    solution = bestSolution
    tabuList = list()
    bestCost = fitnessFunc2(bestSolution, graph)
    counter = 1

    while counter <= iterations:
        neighbours = neighborhood2(solution, graph)
        # print(neighbours)
        found = False # indicates we found a set of different nodes that are not in the tabu list 
        currentBestSolutionIndex = 0
        currentBestSolution = neighbours[currentBestSolutionIndex]

        while not found and currentBestSolutionIndex < len(neighbours) - 1:
            i = 0
            while i < len(currentBestSolution):
                j = 0
                while j < len(currentBestSolution[i]):
                    if currentBestSolution[i][j] != solution[i][j]: # if the same node position is not equal 
                        firstNode = currentBestSolution[i][j]
                        secondNode = solution[i][j]
                        break
                    j += 1
                i += 1

            if [firstNode, secondNode] not in tabuList and [secondNode, firstNode] not in tabuList: # check if the swap is in the tabu list or not 
                tabuList.append([firstNode, secondNode]) # add the set to the tabuList
                found = True 

                solution = currentBestSolution
                cost = fitnessFunc2(currentBestSolution, graph)

                if cost < bestCost: # checks if the new solution is better than the current best 
                    bestCost = cost
                    bestSolution = solution
            
            else:
                currentBestSolutionIndex += 1 # if nothing was found in with the current swap, go the the next one 
                currentBestSolution = neighbours[currentBestSolutionIndex]
        
        if len(tabuList) >= tabuSize:
            tabuList.pop(0) # removes the oldest element in the list

        counter = counter + 1
    
    return bestSolution, bestCost

In [100]:
bestSolution, bestCost = tabuSearch2(pickupNetwork2, 10000, 14, solutions)
print("Start solution: ", solutions, " Cost: ", fitnessFunc2(s, pickupNetwork2))
print("End solution: ", bestSolution, " Cost: ", bestCost)

Start solution:  [[0, 3, 4, 5, 6, 7, 8], [1, 9, 10, 11, 12, 13, 14], [2, 15, 16, 17, 18, 19, 20]]  Cost:  205315.30000000002
End solution:  [[0, 3, 5, 4, 7, 8, 6], [1, 11, 9, 12, 10, 13, 14], [2, 15, 16, 17, 18, 19, 20]]  Cost:  164574.7


### Genetic Algorithm 

In [33]:
class geneticAlgorithm:
    def __init__(self, graph, ps):
        self.graph = graph
        self.populationSize = ps
        self.population = []
        self.fitness = []
        self.normFitness = [999] * ps

        self.initPopulation()
        self.normalizeFitness()
    
    def initPopulation(self):
        for x in range(self.populationSize):
            self.population.append(initSolution(self.graph))
            self.fitness.append(fitnessFunc(self.population[x], self.graph))
    
    def normalizeFitness(self):
        total = 0
        for i in range(self.populationSize):
            total += self.fitness[i]

        for i in range(self.populationSize):
            self.normFitness[i] = self.fitness[i]/total
    
    def pickSolution(self):
        index = 0
        r = random.uniform(0, 1)

        while r > 0:
            r = r - self.normFitness[index]
            index += 1
        
        return self.population[index-1]
    
    def swap(self, solution):
        swapped = False
        while not swapped:
            n1 = random.randint(0, len(self.graph.nodes))
            n2 = random.randint(0, len(self.graph.nodes))

            newSolution = solution.copy()
            newSolution[n1], newSolution[n2] = newSolution[n2], newSolution[n1]
            print(newSolution)
            swapped = checkSolution(newSolution, graph)

        return newSolution
    
    def mutateSolution(self, solution, mutationFactor):
        newSolution = self.swap(solution)
        return newSolution
    
    def findBestSolution(self):
        bestSolution = self.population[0]
        bestCost = self.fitness[0]
        for i in range(1, self.populationSize):
            if self.fitness[i] > bestCost:
                bestCost = self.fitness[i]
                bestSolution = self.population[i]
        
        return bestSolution, bestCost
        
    
    def geneticAlgorithm(self, iterations):
        bestSolution, bestCost = self.findBestSolution()
        for iter in range(iterations):
            newPopultion = []
            newFitness = []
            for s in range(self.populationSize):
                newPopultion.append(self.mutateSolution(self.pickSolution(), 1))
                newFitness.append(fitnessFunc(newPopultion[s], self.graph))
            self.population = newPopultion
            self.fitness = newFitness
            self.normalizeFitness()

            popBestSolution, popBestCost = self.findBestSolution()
            if popBestCost < bestCost:
                bestSolution = popBestSolution
                bestCost = popBestCost
        
        return bestSolution, bestCost

In [34]:
g = geneticAlgorithm(pickupNetwork, 10)

s, c = g.geneticAlgorithm(1000)

print("End solution: ", s, " Cost: ", c)

[0, 1, 3, 2, 4, 5, 6]


TypeError: graph.getEdgeFromEdgeNodes() missing 1 required positional argument: 'node2'

### Genetic Algorithm 

In [44]:
class geneticAlgorithm:
    def __init__(self, graph, ps):
        self.graph = graph
        self.populationSize = ps
        self.population = []
        self.fitness = []
        self.normFitness = [999] * ps

        self.initPopulation()
        self.normalizeFitness()
    
    def initPopulation(self):
        for x in range(self.populationSize):
            self.population.append(initSolution(self.graph))
            self.fitness.append(fitnessFunc(self.population[x], self.graph))
    
    def normalizeFitness(self):
        total = 0
        for i in range(self.populationSize):
            total += self.fitness[i]

        for i in range(self.populationSize):
            self.normFitness[i] = self.fitness[i]/total
    
    def pickSolution(self):
        index = 0
        r = random.uniform(0, 1)

        while r > 0:
            r = r - self.normFitness[index]
            index += 1
        
        return self.population[index-1]
    
    def swap(self, solution):
        swapped = False
        while not swapped:
            n1 = random.randint(0, len(self.graph.nodes))
            n2 = random.randint(0, len(self.graph.nodes))

            newSolution = solution.copy()
            newSolution[n1], newSolution[n2] = newSolution[n2], newSolution[n1]
            print(newSolution)
            swapped = checkSolution(newSolution, graph)

        return newSolution
    
    def mutateSolution(self, solution, mutationFactor):
        newSolution = self.swap(solution)
        return newSolution
    
    def findBestSolution(self):
        bestSolution = self.population[0]
        bestCost = self.fitness[0]
        for i in range(1, self.populationSize):
            if self.fitness[i] > bestCost:
                bestCost = self.fitness[i]
                bestSolution = self.population[i]
        
        return bestSolution, bestCost
        
    
    def geneticAlgorithm(self, iterations):
        bestSolution, bestCost = self.findBestSolution()
        for iter in range(iterations):
            newPopultion = []
            newFitness = []
            for s in range(self.populationSize):
                newPopultion.append(self.mutateSolution(self.pickSolution(), 1))
                newFitness.append(fitnessFunc(newPopultion[s], self.graph))
            self.population = newPopultion
            self.fitness = newFitness
            self.normalizeFitness()

            popBestSolution, popBestCost = self.findBestSolution()
            if popBestCost < bestCost:
                bestSolution = popBestSolution
                bestCost = popBestCost
        
        return bestSolution, bestCost

In [45]:
g = geneticAlgorithm(pickupNetwork, 10)

s, c = g.geneticAlgorithm(1000)

print("End solution: ", s, " Cost: ", c)

[0, 3, 4, 5, 1, 2, 6]
3 0


TypeError: getEdgeFromEdgeNodes() missing 1 required positional argument: 'node2'

### testing area 

In [ ]:

r = requests.get('http://127.0.0.1:5000/route/v1/driving/14.423235598020154,35.91419450996914;14.407218690503381,35.888194056331706?steps=false')

data = r.json()

print(data)

{'code': 'Ok', 'routes': [{'geometry': 'ioezE__`wAi@jEj@rGwKfMe@fC`ClG`XlLtCtChFfL~UxMjFbAbCwAxOpCn[eG`O]pQxC`Ce@bBrBtMxE', 'legs': [{'steps': [], 'distance': 4153.7, 'duration': 312.7, 'summary': '', 'weight': 312.7}], 'distance': 4153.7, 'duration': 312.7, 'weight_name': 'routability', 'weight': 312.7}], 'waypoints': [{'hint': 'SegAgGHoAIAAAAAAbAAAAAAAAAAfAAAAAAAAAN0ClkIAAAAA7aepQQAAAABsAAAAAAAAAB8AAAAgAAAAABTcADQCJALEFNwA0wEkAgAAfxFEWslf', 'distance': 20.706096, 'name': 'Triq Sir Arthur Borton', 'location': [14.42304, 35.914292]}, {'hint': '7MIAgAzDAIAcAAAAXAAAAO0AAAB3AAAASikZQo_98UJuL51DgQkeQxwAAABcAAAA7QAAAHcAAAAgAAAAY9fbAPWbIwIz1tsAQpwjAgYAHwFEWslf', 'distance': 28.75223, 'name': 'Triq tal-Infetti', 'location': [14.407523, 35.888117]}]}


In [ ]:
print(data['routes'][0]['distance'])
print(data['routes'][0]['geometry'])

4153.7
ioezE__`wAi@jEj@rGwKfMe@fC`ClG`XlLtCtChFfL~UxMjFbAbCwAxOpCn[eG`O]pQxC`Ce@bBrBtMxE
